# Using MLflow with Tune

(tune-mlflow-ref)=

:::{warning}
If you are using these MLflow integrations with {ref}`ray-client-ref`, it is recommended that you setup a
remote Mlflow tracking server instead of one that is backed by the local filesystem.
:::

[MLflow](https://mlflow.org/) is an open source platform to manage the ML lifecycle, including experimentation,
reproducibility, deployment, and a central model registry. It currently offers four components, including
MLflow Tracking to record and query experiments, including code, data, config, and results.

```{image} /images/mlflow.png
:align: center
:alt: MLflow
:height: 80px
:target: https://www.mlflow.org/
```

Ray Tune currently offers two lightweight integrations for MLflow Tracking.
One is the {ref}`MLflowLoggerCallback <tune-mlflow-logger>`, which automatically logs
metrics reported to Tune to the MLflow Tracking API.

The other one is the {ref}`@mlflow_mixin <tune-mlflow-mixin>` decorator, which can be
used with the function API. It automatically
initializes the MLflow API with Tune's training information and creates a run for each Tune trial.
Then within your training function, you can just use the
MLflow like you would normally do, e.g. using `mlflow.log_metrics()` or even `mlflow.autolog()`
to log to your training process.

```{contents}
:backlinks: none
:local: true
```

## Running an MLflow Example

In the following example we're going to use both of the above methods, namely the `MLflowLoggerCallback` and
the `mlflow_mixin` decorator to log metrics.
Let's start with a few crucial imports:

In [1]:
import os
import tempfile
import time

import mlflow

from ray import air, tune
from ray.air import session
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.integration.mlflow import mlflow_mixin

Next, let's define an easy objective function (a Tune `Trainable`) that iteratively computes steps and evaluates
intermediate scores that we report to Tune.

In [2]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def easy_objective(config):
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

Given an MLFlow tracking URI, you can now simply use the `MLflowLoggerCallback` as a `callback` argument to
your `RunConfig()`:

In [3]:
def tune_function(mlflow_tracking_uri, finish_fast=False):
    tuner = tune.Tuner(
        easy_objective,
        tune_config=tune.TuneConfig(
            num_samples=5
        ),
        run_config=air.RunConfig(
            name="mlflow",
            callbacks=[
                MLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name="example",
                    save_artifact=True,
                )
            ],
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
        },
    )
    results = tuner.fit()

To use the `mlflow_mixin` decorator, you can simply decorate the objective function from earlier.
Note that we also use `mlflow.log_metrics(...)` to log metrics to MLflow.
Otherwise, the decorated version of our objective is identical to its original.

In [4]:
@mlflow_mixin
def decorated_easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Log the metrics to mlflow
        mlflow.log_metrics(dict(mean_loss=intermediate_score), step=step)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

With this new objective function ready, you can now create a Tune run with it as follows:

In [5]:
def tune_decorated(mlflow_tracking_uri, finish_fast=False):
    # Set the experiment, or create a new one if does not exist yet.
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name="mixin_example")
    
    tuner = tune.Tuner(
        decorated_easy_objective,
        tune_config=tune.TuneConfig(
            num_samples=5
        ),
        run_config=air.RunConfig(
            name="mlflow",
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
            "mlflow": {
                "experiment_name": "mixin_example",
                "tracking_uri": mlflow.get_tracking_uri(),
            },
        },
    )
    results = tuner.fit()

If you hapen to have an MLFlow tracking URI, you can set it below in the `mlflow_tracking_uri` variable and set
`smoke_test=False`.
Otherwise, you can just run a quick test of the `tune_function` and `tune_decorated` functions without using MLflow.

In [6]:
smoke_test = True

if smoke_test:
    mlflow_tracking_uri = os.path.join(tempfile.gettempdir(), "mlruns")
else:
    mlflow_tracking_uri = "<MLFLOW_TRACKING_URI>"

tune_function(mlflow_tracking_uri, finish_fast=smoke_test)
if not smoke_test:
    df = mlflow.search_runs(
        [mlflow.get_experiment_by_name("example").experiment_id]
    )
    print(df)

tune_decorated(mlflow_tracking_uri, finish_fast=smoke_test)
if not smoke_test:
    df = mlflow.search_runs(
        [mlflow.get_experiment_by_name("mixin_example").experiment_id]
    )
    print(df)

2022-07-22 16:27:41,371	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8271
2022-07-22 16:27:43,768	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_d4e29_00000,TERMINATED,127.0.0.1:52551,38,23,4.78039,5,0.549093,4,-4.78039
easy_objective_d4e29_00001,TERMINATED,127.0.0.1:52561,86,88,8.87624,5,0.548692,4,-8.87624
easy_objective_d4e29_00002,TERMINATED,127.0.0.1:52562,22,95,2.45641,5,0.587558,4,-2.45641
easy_objective_d4e29_00003,TERMINATED,127.0.0.1:52563,11,81,1.3994,5,0.560393,4,-1.3994
easy_objective_d4e29_00004,TERMINATED,127.0.0.1:52564,21,27,2.94746,5,0.534,4,-2.94746


2022-07-22 16:27:44,945	INFO plugin_schema_manager.py:52 -- Loading the default runtime env schemas: ['/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].


Result for easy_objective_d4e29_00000:
  date: 2022-07-22_16-27-47
  done: false
  experiment_id: 421feb6ca1cb40969430bd0ab995fe37
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.8
  neg_mean_loss: -13.8
  node_ip: 127.0.0.1
  pid: 52551
  time_since_restore: 0.00015282630920410156
  time_this_iter_s: 0.00015282630920410156
  time_total_s: 0.00015282630920410156
  timestamp: 1658503667
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d4e29_00000
  warmup_time: 0.0036253929138183594
  
Result for easy_objective_d4e29_00000:
  date: 2022-07-22_16-27-48
  done: true
  experiment_id: 421feb6ca1cb40969430bd0ab995fe37
  experiment_tag: 0_height=38,width=23
  hostname: Kais-MacBook-Pro.local
  iterations: 4
  iterations_since_restore: 5
  mean_loss: 4.780392156862745
  neg_mean_loss: -4.780392156862745
  node_ip: 127.0.0.1
  pid: 52551
  time_since_restore: 0.5490927696228027
  time_this_iter_s: 0.12111282348632812
  time_total_

2022-07-22 16:27:51,033	INFO tune.py:738 -- Total run time: 7.27 seconds (6.28 seconds for the tuning loop).
2022/07/22 16:27:51 INFO mlflow.tracking.fluent: Experiment with name 'mixin_example' does not exist. Creating a new experiment.


Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
decorated_easy_objective_d93b6_00000,TERMINATED,127.0.0.1:52581,45,51,4.96729,5,0.460993,4,-4.96729
decorated_easy_objective_d93b6_00001,TERMINATED,127.0.0.1:52598,44,94,4.65907,5,0.434945,4,-4.65907
decorated_easy_objective_d93b6_00002,TERMINATED,127.0.0.1:52599,93,25,10.2091,5,0.471808,4,-10.2091
decorated_easy_objective_d93b6_00003,TERMINATED,127.0.0.1:52600,40,26,4.87719,5,0.437302,4,-4.87719
decorated_easy_objective_d93b6_00004,TERMINATED,127.0.0.1:52601,16,65,1.97037,5,0.468027,4,-1.97037


Result for decorated_easy_objective_d93b6_00000:
  date: 2022-07-22_16-27-54
  done: false
  experiment_id: 2d0d9fbc13c64acfa27153a5fb9aeb68
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 14.5
  neg_mean_loss: -14.5
  node_ip: 127.0.0.1
  pid: 52581
  time_since_restore: 0.001725912094116211
  time_this_iter_s: 0.001725912094116211
  time_total_s: 0.001725912094116211
  timestamp: 1658503674
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d93b6_00000
  warmup_time: 0.20471811294555664
  
Result for decorated_easy_objective_d93b6_00000:
  date: 2022-07-22_16-27-54
  done: true
  experiment_id: 2d0d9fbc13c64acfa27153a5fb9aeb68
  experiment_tag: 0_height=45,width=51
  hostname: Kais-MacBook-Pro.local
  iterations: 4
  iterations_since_restore: 5
  mean_loss: 4.9672897196261685
  neg_mean_loss: -4.9672897196261685
  node_ip: 127.0.0.1
  pid: 52581
  time_since_restore: 0.46099305152893066
  time_this_iter_s: 0.1098420619964599

2022-07-22 16:27:58,211	INFO tune.py:738 -- Total run time: 7.15 seconds (7.01 seconds for the tuning loop).


This completes our Tune and MLflow walk-through.
In the following sections you can find more details on the API of the Tune-MLflow integration.

## MLflow AutoLogging

You can also check out {doc}`here </tune/examples/includes/mlflow_ptl_example>` for an example on how you can
leverage MLflow auto-logging, in this case with Pytorch Lightning

## MLflow Logger API

(tune-mlflow-logger)=

```{eval-rst}
.. autoclass:: ray.air.integrations.mlflow.MLflowLoggerCallback
   :noindex:
```

## MLflow Mixin API

(tune-mlflow-mixin)=

```{eval-rst}
.. autofunction:: ray.tune.integration.mlflow.mlflow_mixin
   :noindex:
```

## More MLflow Examples

- {doc}`/tune/examples/includes/mlflow_ptl_example`: Example for using [MLflow](https://github.com/mlflow/mlflow/)
  and [Pytorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning) with Ray Tune.